In [1]:
!pip3 install transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [16]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import re
import textwrap


In [17]:
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [9]:
def generate_story(prompt, model, tokenizer, device, max_length=500):
    """
    Function to generate a story based on a given prompt.

    Parameters:
    - prompt: Initial text to generate the story from.
    - model: The GPT-2 model to use for story generation.
    - tokenizer: The tokenizer corresponding to the GPT-2 model.
    - device: Device on which the model will run (GPU/CPU).
    - max_length: Maximum length of the generated text.

    Returns:
    - story: Generated story as text.
    """
    # Encode the prompt text into tokens
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)

    # Create an attention mask
    attention_mask = torch.ones(inputs.shape, device=device)

    # Generate a story with the attention mask
    outputs = model.generate(
        inputs,
        max_length=max_length,  # Maximum length of the generated story
        num_return_sequences=1,  # Number of stories to generate
        no_repeat_ngram_size=2,  # Prevent repeating phrases
        do_sample=True,  # Use sampling to increase creativity
        top_k=50,  # Sampling: Consider top 50 words
        top_p=0.95,  # Sampling: Consider top probability
        temperature=0.7,  # Creativity control
        pad_token_id=tokenizer.eos_token_id,  # End of sequence token
        attention_mask=attention_mask  # Pass the attention mask here
    )

    # Decode the generated story from tokens to text
    story = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return story


In [18]:
def clean_story(story, max_line_length=80):
    """
    Function to clean and format the generated story into readable paragraphs.

    Parameters:
    - story: Generated story text.
    - max_line_length: Maximum length of lines for readability.

    Returns:
    - wrapped_story: Cleaned and wrapped story text.
    """
    # Remove unwanted line breaks and extra spaces
    story = story.replace("\n", " ").replace("  ", " ").strip()

    # Split the story into sentences
    sentences = re.split(r'(?<=[.!?]) +', story)

    # Format the sentences into paragraphs, where each paragraph contains 3-5 sentences
    paragraphs = []
    paragraph = []
    for sentence in sentences:
        paragraph.append(sentence)

        # Add paragraph after 3-5 sentences
        if len(paragraph) >= 3:  # You can change this number for more/less sentences per paragraph
            paragraphs.append(" ".join(paragraph))
            paragraph = []

    # If there are remaining sentences, add them as the last paragraph
    if paragraph:
        paragraphs.append(" ".join(paragraph))

    # Join the paragraphs with a blank line between them
    formatted_story = "\n\n".join(paragraphs)

    # Wrap each paragraph to a max line length for readability
    wrapped_story = "\n\n".join([textwrap.fill(paragraph, width=max_line_length) for paragraph in paragraphs])

    return wrapped_story


In [20]:
def generate_title_from_story(story, model, tokenizer, device, max_length=30):

    prompt = (
        "Generate a creative and meaningful title for the following story:\n\n"
        f"{story}\n\nTitle:"
    )

    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    attention_mask = torch.ones(inputs.shape, device=device)

    outputs = model.generate(
        inputs,
        max_length=inputs.shape[1] + max_length,
        num_return_sequences=1,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.8,
        pad_token_id=tokenizer.eos_token_id,
        attention_mask=attention_mask
    )

    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract title after "Title:"
    match = re.search(r"Title:\s*(.*)", full_output)
    if match:
        title = match.group(1).split('\n')[0].strip()
        return f" {title}"
    else:
        return " Untitled Story"



In [23]:
# Define the prompt to start the story
prompt = "A robot named Max in a desert..."

# Generate the story
story = generate_story(prompt, model, tokenizer, device)

# Clean and format the story with wrapping
formatted_story = clean_story(story, max_line_length=100)  # Clean and format the story with wrapping

title = generate_title_from_story(formatted_story, model, tokenizer, device)
print(title)
print("\n" +  "\n")

# Print the formatted story
print(formatted_story)




 The Doctor's dream



A robot named Max in a desert... A computer named Nick in the desert. (It is said that Nick was born
in this world, but he was not a human.) , a robot called Maxin in an abandoned bunker in "Diary of a
Time Traveler".

and a computer called Nick. (Nick was a supercomputer.) "A giant robot... the one you can't live
with, or have a real life with.

It's one of those things that you don't really want to live in." . "I've been thinking about it for
a while now... and I think it's time to get serious.

I'm about to go get some sleep. That's the reason I put my computer in my room. No, I want this to
be a peaceful, happy, peaceful time." - Nick's dream (src) - Nick is the only human on a planet with
a civilization that doesn't have to worry about humans.

He is known to have visited some of the planets that have been visited by the aliens, and he is also
known as the "One Man Planet". He has also been known not to bother the humans at all, except in
certain areas, such